In [1]:
import tensorflow as tf
import os

2025-08-20 13:04:47.928628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755687887.947649  135547 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755687887.952539  135547 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755687887.965486  135547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755687887.965504  135547 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755687887.965506  135547 computation_placer.cc:177] computation placer alr

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
MODEL_PATH = "../model/2025-07-14/cnn_1d_model.keras"
model = tf.keras.models.load_model(MODEL_PATH, compile=False)

2025-08-20 13:04:51.994304: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-08-20 13:04:51.994334: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:167] env: CUDA_VISIBLE_DEVICES="-1"
2025-08-20 13:04:51.994339: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-08-20 13:04:51.994343: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-08-20 13:04:51.994347: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: mThinkPad-t480
2025-08-20 13:04:51.994350: I external/local_xla/xla/stream_executor/cud

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [5]:

converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Jeśli chcesz wymusić float16 zamiast float32 (mniejsze rozmiary i szybsze na CPU)
converter.target_spec.supported_types = [tf.float16]


In [6]:
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmpmsystndc/assets


INFO:tensorflow:Assets written to: /tmp/tmpmsystndc/assets


Saved artifact at '/tmp/tmpmsystndc'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 63), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 36), dtype=tf.float32, name=None)
Captures:
  140308235166416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235167952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235166608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235167376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235170256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235171024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235171216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140308235169872: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1755687899.226358  135547 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1755687899.226381  135547 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-20 13:04:59.226788: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpmsystndc
2025-08-20 13:04:59.227308: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-20 13:04:59.227315: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpmsystndc
I0000 00:00:1755687899.232352  135547 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-08-20 13:04:59.233465: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-20 13:04:59.260185: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpmsystndc
2025-08-20 13:04:59.268255: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 

In [7]:
TFLITE_PATH = "cnn1d.tflite"
with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print(f"Model TFLite zapisany: {TFLITE_PATH}")


Model TFLite zapisany: cnn1d.tflite


In [4]:
import cv2
import time
import numpy as np
import mediapipe as mp
import tensorflow as tf

# --- Ustawienia ---
SEQ_LENGTH = 30
THRESHOLD = 0.8
COOLDOWN_SECONDS = 1.5

# Ścieżki do modelu i etykiet
TFLITE_MODEL_PATH = "cnn1d.tflite"
with open('../model/2025-07-14/labels.txt', 'r') as f:
    LABELS = [line.strip() for line in f.readlines()]

# --- Inicjalizacja MediaPipe ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# --- Wczytanie modelu TFLite ---
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- Funkcje pomocnicze ---
def extract_hand_landmarks(results):
    if results.multi_hand_landmarks:
        hand = results.multi_hand_landmarks[0]
        return np.array([[lm.x, lm.y, lm.z] for lm in hand.landmark])
    return None

def normalize_sequence(sequence_array):
    sequence_array = np.array(sequence_array)
    origin = sequence_array[0][0]  # nadgarstek 1. klatki
    return sequence_array - origin

def predict_tflite(sequence):
    norm_seq = normalize_sequence(sequence)
    input_data = norm_seq.reshape(1, SEQ_LENGTH, 63).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]
    max_prob = np.max(output_data)
    label = LABELS[np.argmax(output_data)]
    return label, max_prob

# --- Inicjalizacja zmiennych ---
sequence = []
recognized_text = ""
last_prediction_time = 0.0

# --- Kamera ---
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    hand_landmarks = extract_hand_landmarks(results)
    current_time = time.time()
    cooldown_remaining = max(0, COOLDOWN_SECONDS - (current_time - last_prediction_time))

    if hand_landmarks is not None:
        sequence.append(hand_landmarks)
        if len(sequence) == SEQ_LENGTH:
            label, max_prob = predict_tflite(sequence)
            if max_prob > THRESHOLD and cooldown_remaining == 0:
                recognized_text += label
                last_prediction_time = current_time
            sequence.pop(0)
    else:
        sequence = []

    # --- Overlay MediaPipe i predykcja ---
    if results.multi_hand_landmarks:
        for hand_landmarks_mp in results.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                frame, hand_landmarks_mp, mp_hands.HAND_CONNECTIONS
            )

    # Pasek cooldownu
    bar_x, bar_y, bar_width, bar_height = 10, 80, 300, 20
    progress = int((1 - cooldown_remaining / COOLDOWN_SECONDS) * bar_width)
    bar_color = (0, 255, 0) if cooldown_remaining == 0 else (0, 0, 255)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (255, 255, 255), 2)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + progress, bar_y + bar_height), bar_color, -1)
    cv2.putText(frame, f'Cooldown: {cooldown_remaining:.1f}s', (bar_x, bar_y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, bar_color, 2)

    # Napis z rozpoznanym tekstem
    cv2.putText(frame, f'Text: {recognized_text}', (10, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    cv2.imshow('Gesture Recognition TFLite', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1755688100.040368  135789 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755688100.061950  135789 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
